# Preparing Results CSV Files Before Plotting
**Table of Contents**
- [Preparing Results CSV Files Before Plotting](#preparing-results-csv-files-before-plotting)
- [Import Required Libraries](#import-required-libraries)
- [Load Results CSV Data](#load-results-csv-data)
- [Case 1: Extracting Information from Standard Model Filenames](#case-1-extracting-information-from-standard-model-filenames)
- [Case 2: Manual Column Creation (For Custom Filenames)](#case-2-manual-column-creation-for-custom-filenames)
- [(Optional) Merging with Model Metrics](#optional-merging-with-model-metrics)

Before using `AnalysisPlotter` to generate insightful graphs, it is necessary to ensure that your results file contains all required columns. This notebook helps you prepare those columns.

If you want to plot energy consumption **versus model performance metrics** (e.g., accuracy, F1-score), you must add your metric of choice as a column.



---

## Import Required Libraries

In [ ]:
import sys
import os

lib_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
if lib_path not in sys.path:
    sys.path.append(lib_path)

import pandas as pd
from PruneEnergyAnalizer import parse_model_name


---

## Load Results CSV Data

In [ ]:
# Load the CSV file
results_df = pd.read_csv("results.csv")
results_df


---

## Case 1: Extracting Information from Standard Model Filenames

In [ ]:
results_df = parse_model_name(results_df)
results_df[["MODEL_NAME", "GPR", "Architecture", "Pruning Distribution"]]



---

## Case 2: Manual Column Creation (For Custom Filenames)

In [ ]:
# Read the metadata file
metadata_df = pd.read_csv('metadata.csv')  # must contain MODEL_NAME, Architecture, Pruning Distribution

# Merge based on MODEL_NAME
results_df = pd.merge(results_df, metadata_df, on='MODEL_NAME', how='left')

# Uncomment if you want to set a default value for "Pruning Distribution" becouse it is not present in the metadata
# merged_df["Pruning Distribution"] = 'PD0' 

# Display the merged DataFrame
results_df.head()



---

## (Optional) Merging with Model Metrics

In [ ]:
# Read the metadata file
metric_df = pd.read_csv('metadata.csv')  # must contain MODEL_NAME and  YOUR_METRIC

# Merge based on MODEL_NAME
results_df = pd.merge(results_df, metric_df, on='MODEL_NAME', how='left')

# Display the merged DataFrame
results_df.head()


---

## Save Prepared Results to CSV

After all necessary processing, you can save your prepared DataFrame (e.g., `results_df`) to a new CSV file for later use or for plotting.


In [ ]:
# Save the processed results DataFrame to a new CSV file
results_df.to_csv("prepared_results.csv", index=False)
print("Results saved to prepared_results.csv")
